# SAR Training Workshop for Forest Degradation

# PART 5 - Cumulative Sum Classifier

Josef Kellndorfer, Ph.D., President and Senior Scientist, Earth Big Data, LLC

Revision date: June 2020 Virtual Training

In this chapter we are looking at the test site in Peru. We will study segmentation of the time series into a baseline and monitoring period. For the baseline period we will generate a crude forest mask based on the COV thresholding method. This mask will be applied to study the monitoring period with cumluative sums of residuals agains a time series mean.  

We explore the use of  Change Point Detection with *Cumulative Sum*. For details please see the EBD_SAR4 Notebook in the servor


# Import Python modules

In [ ]:
import os,sys
sys.path.append('/notebooks/github/servir_training/notebooks_202006/Nadia_Exo')
import ebdpy as ebd
import glob

from skimage import exposure # to enhance image display

import numpy as np
import pandas as pd
import datetime as dt
import gdal
import xarray as xr

%matplotlib inline
import matplotlib.pylab as plt
import matplotlib.patches as patches  # Needed to draw rectangles
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# In case changes are made to the ebdpy module, reload it
import importlib
importlib.reload(ebd)

In [ ]:
from dask.distributed import Client
client=Client()
client

# Select the Project data set and time series data

In [ ]:
PLOT=True  # Change to False when one wants to only run the notebook to generate an output geotif.
datapath='/Data/Sentinel-1/Peru/18MVT'

userdatapath='/Userdata'
os.makedirs(userdatapath,exist_ok=True)
os.chdir(datapath)

# List the files in the Data directory:


In [ ]:
files=glob.glob(os.path.join(datapath,"*"))

In [ ]:
ts_stacks=[x for x in files if x.endswith('_subset.tif') if not x.find('tsmetrics')>-1]
ts_metrics=[x for x in files if x.endswith('_tsmetrics_subset.tif')]
ts_stacks.sort()
ts_metrics.sort()
for i in zip(['ts_stacks','ts_metrics'],[ts_stacks,ts_metrics]):
    print(i[0])
    for j in i[1]:
        print(i[1].index(j),j)

# Let's pick a time series stack index for a likepol and crosspol image name

In [ ]:
xp=2
lp=3
likepol=ts_stacks[lp]
crosspol=ts_stacks[xp]

# Choose a polarization for the working data array

In [ ]:
ds_source=crosspol
dsname=os.path.splitext(os.path.basename(ds_source))[0].replace('_mtfil','').replace('_subset','')
dsname

## Read the SAR Time Series data stack
During reading, we are building a xarray dataset setting the time series index from the description attribute. For details take a look at the source code of `ebdpy`.

In [ ]:
da = ebd.read_sar(ds_source,dsname)

In [ ]:
# Subset the data with coordinates xmin,ymin,xmax,ymax
ul=[458152.776,9286360.706]

lr=[]
sizemeters=8000 #this is a buffer from the point (unit E?)Original was 15000
if lr:
    subset=[ul[0],ul[1],lr[0],lr[1]]
else:
    subset=[ul[0],ul[1],ul[0]+sizemeters,ul[1]-sizemeters]
da=ebd.da_subset_xy(da,subset)

In [ ]:
da

In [ ]:
ts_splitpoint='2019-01-14'
da_baseline=da.sel(time=slice('2017-05-24',ts_splitpoint))
da=da.sel(time=slice(ts_splitpoint,'2020-04-14')).chunk({'x':-1,'y':-1})


In [ ]:
da_baseline

## Let's make a crude forest/non-forest mask for the pre period from COV thresholding

In [ ]:
cov=da_baseline.std('time')/da_baseline.mean('time')

In [ ]:
hv.Image(cov,kdims=['x','y']).opts(cmap='viridis',width=600,height=600,tools=['hover']) 

In [ ]:
cov_thres=0.14
forest_mask=cov<cov_thres
forest_mask.plot(cmap='Greens')

Let's determine a threshold from the plot above to make a mask and look at it.

In [ ]:
cov_thres=0.14
forest_mask=cov<cov_thres
forest_mask.plot(cmap='Greens')

Add the mask (forest mask) to the data array ...

In [ ]:
da.coords['mask'] = (('y', 'x'), forest_mask) 

... and apply it on the data array itself.

In [ ]:
da = da.where(da.mask)

Note: We could skip the step of adding the mask to the data array and supply forest_mask straight to the da.where() function.

    da = da.where(forest_mask)

Check with a plot if this worked

In [ ]:
da.mean('time').plot()

## RGB Plot of Time Series Max/Min Difference from the Mean

Let's make a plot of time series metrics where we take the difference with the mean of the maximum and minimum backscatter values. We generate three RGB bands for that from the time series stack Mean, Mean-Min, and Max-Mean

In [ ]:
g=da.mean('time')
r=g-da.min('time')
b=da.max('time')-g
rgb=np.dstack([r.values,g.values,b.values])
# Let's histogram equalize the data
rgb_stretched=ebd.rgb_stretch(rgb)

In [ ]:
ebd.rgb_plot(rgb_stretched,da,f"RGB Time Series Mean-Min/Mean/Max-Mean ({dsname})")

### Finding:

Looks like the areas for degradation can be identified. Let's see if we can find a classifier for it. 

## Select the time series segment to analyze

Let's see if we can find trends of change by comparing the minimum SAR backscatter in each quarter.

In [ ]:
# To work with a subsetted resampled stack uncomment here
# resampler = da.resample(time='36D',keep_attrs=True)
# mymetric='median'
# qmetric=resampler.median(keep_attrs=True).chunk({'x':-1,'y':-1})

In [ ]:
# Work with the entire stack
mymetric='all'
qmetric=da.chunk({'x':-1,'y':-1})

In [ ]:
qmetric.plot(col='time',col_wrap=5,cmap='viridis',vmin=0.01,vmax=0.09,xticks=[],yticks=[])

# Let's look at Seasonality

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(12,8))
qmetric.mean(dim=['x','y']).plot(ax=ax,label='mean')
qmetric.min(dim=['x','y']).plot(ax=ax,label='min')
#qmetric.max(dim=['x','y']).plot(ax=ax,label='max')
q=[0.02,.98]
pl,ph=qmetric.quantile(q,dim=['x','y'])
pl.plot(ax=ax,label=f'p{q[0]*100:.0f}')
ph.plot(ax=ax,label=f'p{q[1]*100:.0f}')
ax.set_title(f'Time series means of quarterly metric "{mymetric}"')
fig.legend()

## Compute the Residuals of the resampled metric to the mean of the metric

In [ ]:
qmetric.mean('time').plot(vmin=0.01,vmax=0.1,figsize=(12,10))

In [ ]:
qmetric_residuals=qmetric-qmetric.mean('time').persist()

In [ ]:
qmetric_residuals.plot(col='time',col_wrap=4,figsize=(16,20),vmin=-0.02,vmax=0.02,cmap='RdBu_r')

In [ ]:
S = qmetric_residuals.cumsum('time')

In [ ]:
S

In [ ]:
pt=[459660.561,9285273.414]
pt=[459321.414,9283179.227]

Spt= S.sel({'x':pt[0],'y':pt[1]},method='nearest').persist()

In [ ]:
fig,ax=plt.subplots(figsize=(12,6))
Spt.plot(ax=ax)
ax.set_title(f'S Curve at Point {pt}')
ax.set_ylabel('Cumulative Sum of Residual from Mean')

In [ ]:
#S.plot(col='time',col_wrap=4,figsize=(16,18))

In [ ]:
Smax=S.where(S.mask).max('time')
Smin=S.where(S.mask).min('time')
SDiff=Smax-abs(Smin)
Smax.plot(figsize=(12,12),cmap='viridis')
Smin.plot(figsize=(12,12),cmap='viridis')
SDiff.plot(figsize=(12,12),cmap='viridis')


In [ ]:
hv.Image(SDiff,kdims=['x','y']).opts(width=700,height=700,cmap='viridis',tools=['hover'])

## Setting a threshold for SDiff as classifier

We can pick a threshold from the interactive plot. Or:

Because we applied a forest mask to our subsetted data stack, we can determine a threshold for disturbance as a multiple of the standard deviation of the mean as we assume Gaussian Distribution in the SDiff pdf:

$thres = \overline{S_{Diff}} + x \times \sigma_{S_{Diff}}$

with 
$x$  factor for $\sigma$ multiplication from the mean.

Let's plot the histogram to confirm a quasi Gaussian pdf for SDiff.

In [ ]:
_=plt.hist(SDiff.values.flatten(),bins=200)
_=plt.title('Histogram of SDiff')

In [ ]:
x=2.5
dthres=float(SDiff.mean()+x*SDiff.std())
ndthres=float(SDiff.mean()-x*SDiff.std())
# print(f'Setting the threshold for SDiff to {x}*Std.Dev.: {dthres:.2f}')
print(f'Setting the threshold for SDiff to {x}*Std.Dev.: {ndthres:.2f} {dthres:.2f}')

### Applying the threshold to the mask

In [ ]:
# disturbance=SDiff>dthres
disturbance=np.logical_or(SDiff>dthres, SDiff<ndthres)

### Plot the mask

In [ ]:
plt.figure(figsize=(8,8))
disturbance.plot(cmap='Reds',figsize=(10,8)) #change 'Reds' for 'Gray'
_=plt.title(f'Forest Disturbance after {ts_splitpoint} from CumSum Classifier. SDiff Threshold: {dthres:.2f}')

## Next steps to perform

We are skipping for this workshop the bootstrapping validation steps for the validation of the points. 
See the Notebook EBD_SAR4 from the 2019 Training in Colombia and Peru. Also in that Notebook are the steps for finding the change date.

## Write the result to disk

In [ ]:
Name=os.path.join(userdatapath,f'{dsname}_CumSum_disturbance_Loreto-Peru_{dthres*10000:.0f}.tif')

In [ ]:
GeoT=ebd.transform2geotras(da.transform)
GeoT

In [ ]:
# Create the tif
ebd.CreateGeoTiff(Array=disturbance.values,Name=Name,GeoT=GeoT,ref_image=ds_source,overwrite=True)
print(gdal.Info(Name))

# Exercises

- Change test sites
- Look at the effect of using cross-polarized versus like-polarized polarizations 
- find other time series subsets e.g. build a mask for 2017 and process only for 2018